## 14. tf data api 정리

- https://m.blog.naver.com/euue717/222086046496

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import datetime
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

# Options for pandas
pd.options.display.max_columns = 150

In [2]:
import tensorflow as tf
import tensorflow.keras as keras # 케라스 쓸때 반드시 이것 추가... 주의https://stackoverflow.com/questions/72409779/modulenotfounderror-no-module-named-tensorflow-keras-i-tried-almost-everyth

- tf.data는 데이터 입력 파이프 라인 빌드를 위한 텐서플로우의 서브패키지, 혹은 다른 말로 API이다. 로컬 파일이나 메모리에 올려져 있는 데이터를 모델에 집어넣기 적합한 텐서로 변환하는 작업을 한다.    
- 하위 tf.data.dataset 은 tf.data의 추상 클래스로써 데이터의 병렬 처리가 용이한 형태, 즉 GPU가 연산이 끝나면 다음 데이터를 바로바로 가져다가(Pre-Fetch) 빠르게 처리할 수 있도록 고안되었다.  
- Numpy나 Pandas, 혹은 Tensorflow 2.0 부터된 아예 통합된 Keras 등 모델에 집어넣기 위해 데이터 전처리(Pre-Processing) 용도로 자주 쓰이고 그만큼 유명한 여러 모듈들이 있다. 하지만 텐서플로우에서 제공하는 tf.data.dataset 클래스가 성능적으로는 가장 최적화되어 있다고 볼 수 있다. 원래는 텐서플로우에서 제공하는 클래스는 아니고 Contributor가 제공한 것이지만, 1.8인가서부터는 공식적으로 제공되고 있다.

### 사용법

설계한 모델에 데이터를 집어넣기 위해서는 메모리나 로컬에 존재하는 배열이나 리스트 같은 데이터를 tf.data.Dataset 객체로 변환해야한다. 머신 러닝에서 흔히 쓰이는 일련된 데이터 묶음인 CSV(엑셀에서 흔히 쓴다), Numpy 배열 혹은 이미지나 오디오 데이터를 소스로 Dataset를 생성하면 된다.

In [9]:
def printDs(ds, take=5):
    for ex in ds.take(take):
        print(ex)

1) from_tensors

In [5]:
data0 = tf.data.Dataset.from_tensors(np.array(range(5)))

In [6]:
data0

<TensorDataset shapes: (5,), types: tf.int32>

- 1차원 상수를 받아 5크기의 텐서가 저장됨

In [10]:
printDs(data0)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)


- 한개 element만 있다.

2) from_tensor_slices

In [7]:
data1 = tf.data.Dataset.from_tensor_slices(np.array(range(5)))

In [8]:
data1

<TensorSliceDataset shapes: (), types: tf.int32>

In [11]:
printDs(data1)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


- 크기가 1인 텐서 5개로 쪼개짐

In [17]:
rand_data = tf.random.uniform([4, 10])

In [18]:
rand_data

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[0.42815912, 0.00710964, 0.3391682 , 0.4658395 , 0.93045926,
        0.92477214, 0.65005493, 0.6798675 , 0.13161397, 0.4779011 ],
       [0.75340486, 0.20953739, 0.38959002, 0.8471327 , 0.94131374,
        0.36750066, 0.2918458 , 0.56763077, 0.50440013, 0.1936599 ],
       [0.2197808 , 0.9261099 , 0.27181804, 0.45327055, 0.33639467,
        0.05484879, 0.7402874 , 0.4730903 , 0.18069291, 0.3343612 ],
       [0.55200195, 0.40253496, 0.1013273 , 0.1842655 , 0.99668753,
        0.5639099 , 0.7377974 , 0.12682056, 0.52265275, 0.9695517 ]],
      dtype=float32)>

In [19]:
rand_data.shape

TensorShape([4, 10])

In [20]:
data2 = tf.data.Dataset.from_tensor_slices(rand_data)

In [21]:
data2

<TensorSliceDataset shapes: (10,), types: tf.float32>

- 크기가 10인 텐서 4개로 나누어짐

In [22]:
printDs(data2)

tf.Tensor(
[0.42815912 0.00710964 0.3391682  0.4658395  0.93045926 0.92477214
 0.65005493 0.6798675  0.13161397 0.4779011 ], shape=(10,), dtype=float32)
tf.Tensor(
[0.75340486 0.20953739 0.38959002 0.8471327  0.94131374 0.36750066
 0.2918458  0.56763077 0.50440013 0.1936599 ], shape=(10,), dtype=float32)
tf.Tensor(
[0.2197808  0.9261099  0.27181804 0.45327055 0.33639467 0.05484879
 0.7402874  0.4730903  0.18069291 0.3343612 ], shape=(10,), dtype=float32)
tf.Tensor(
[0.55200195 0.40253496 0.1013273  0.1842655  0.99668753 0.5639099
 0.7377974  0.12682056 0.52265275 0.9695517 ], shape=(10,), dtype=float32)


### 기타

tf.data.Dataset은 기본적으로 파이썬의 반복 가능 객체이다. 즉 iterator로 꺼내쓸 수 있다.

In [24]:
for i in data2:
    print(i)
    print('')

tf.Tensor(
[0.42815912 0.00710964 0.3391682  0.4658395  0.93045926 0.92477214
 0.65005493 0.6798675  0.13161397 0.4779011 ], shape=(10,), dtype=float32)

tf.Tensor(
[0.75340486 0.20953739 0.38959002 0.8471327  0.94131374 0.36750066
 0.2918458  0.56763077 0.50440013 0.1936599 ], shape=(10,), dtype=float32)

tf.Tensor(
[0.2197808  0.9261099  0.27181804 0.45327055 0.33639467 0.05484879
 0.7402874  0.4730903  0.18069291 0.3343612 ], shape=(10,), dtype=float32)

tf.Tensor(
[0.55200195 0.40253496 0.1013273  0.1842655  0.99668753 0.5639099
 0.7377974  0.12682056 0.52265275 0.9695517 ], shape=(10,), dtype=float32)

